In [1]:
#imports
#https://stackoverflow.com/questions/1549641/how-can-i-capitalize-the-first-letter-of-each-word-in-a-string
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import time
import random
import sys
pd.set_option('display.max_colwidth', None)

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from scipy import sparse
from matplotlib import pyplot as plt
import utils as ut

In [2]:
books_df, books_rec, movies_df, movies_rec, vg_df, vg_rec = pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl'), pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl'), pd.read_pickle('./pickles/videog_look.pkl'), pd.read_pickle('./pickles/videog_rec.pkl')

In [3]:
df_list = [books_df, books_rec, movies_df, movies_rec, vg_df, vg_rec]

In [4]:
for df in df_list:
    print(df.shape)
    print(ut.size_in_gb(df))

(1489354, 7)
0.406603011 GB
(46575, 46575)
0.422205132 GB
(4405432, 7)
1.166293371 GB
(72385, 72385)
1.843095412 GB
(1648136, 7)
0.436016768 GB
(15938, 15938)
0.076510705 GB


In [123]:
#def choose_item(question):

#select category based on user input:

category = input('Would you like to search for "video games", "movies", or "books"? Please enter one option: ')
while category.lower() not in ['video games', 'movies', 'books']:
    print('Sorry, you need to enter "video games", "movies", or "books"') #error message
    time.sleep(1) #wait 1 second, then make input available for user to try again
    category = input('Would you like to search for "video games", "movies", or "books"? Please enter one option: ')
if category.lower() == 'video games':
    print('Okay, video game recommendations!') #vg_df
    lookup, recommender = vg_df, vg_rec
elif category.lower() == 'movies':
    print('Okay, movie recommendations!') #movie_df
    lookup, recommender = movies_df, movies_rec
elif category.lower() == 'books':
    print('Okay, book recommendations!') #books_df
    lookup, recommender = books_df, books_rec
else:
    print("Sorry, that wasn't one of the options")

#provide input option for search parameters
query = input('Please enter search term; the more specific your term is, the more accurate the results will be!')
wout = input('''Would you like to add a keyword to filter OUT of your results?
                If so, please enter that keyword or phrase here;
                If not, simply enter "no"''')
try:
    query=query.title() #uppercase first letter of each word in the query in case it's not entered that way
    titles = list(lookup[lookup['product_title'].str.contains(query)]['product_title'])
    q = titles[0] #this is the item to search for
    print(f'Recommending items similar to: {q}')
    print(f"""
    {q} has {round(lookup[lookup['product_title']==q]['tot_prod_reviews'].mean())} reviews
    and a {round(lookup[lookup['product_title']==q]['avg_prod_stars'].mean(), 2)} average star rating""")
    print()
    #print('Here are the 10 recommended items for you based on your search parameters: ')
    #print(len(titles))
    if wout.lower() == 'no':
        print('Here are the 10 recommended items for you based on your search parameters: ')
        query_dict = dict(recommender.loc[q].sort_values())
        print(len(query_dict))
        for key in list(query_dict.keys())[1:11]:
            print()
            print(key)
            print(f"""
    This item has {round(lookup[lookup['product_title']==key]['tot_prod_reviews'].mean())} reviews
    and a {round(lookup[lookup['product_title']==key]['avg_prod_stars'].mean(), 2)} average star rating""")
        #print(recommender.loc[titles[0]].sort_values()[1:11]) #1st matching item will be itself so start at 2nd
    else:
        print(f'Here are the 10 recommended items for you based on your search EXCLUDING "{wout}": ')
        wout = wout.title() #capitlize first letters
        query_dict = dict(recommender.loc[q].sort_values())
        filtered_query = [] #make empty list
        for key, value in query_dict.items(): #index into dictionary of results
            if wout not in key: #check if avoided keyword is in results
                filtered_query.append((key, value)) #make list of results that DON'T include "wout" keyword
        for item in filtered_query[1:11]:
            print()
            print(item[0])
            print(f"""
    This item has {round(lookup[lookup['product_title']==item[0]]['tot_prod_reviews'].mean())} reviews
    and a {round(lookup[lookup['product_title']==item[0]]['avg_prod_stars'].mean(), 2)} average star rating""")
        #print([item for item in filtered_query[1:11]])      #return top 10
        
except:
    print(f'Sorry, "{query}" does not appear to be in the product database')
#list of titles that contain given keyword
#print('Here are the 10 recommended items for you based on your search parameters: ')
#print(len(titles))
#print(recommender.loc[titles[0],:].sort_values()[1:11]) #first matching item will be itself so start at second


Would you like to search for "video games", "movies", or "books"? Please enter one option:  video games


Okay, video game recommendations!


Please enter search term; the more specific your term is, the more accurate the results will be! rogue squadron
Would you like to add a keyword to filter OUT of your results?
                If so, please enter that keyword or phrase here;
                If not, simply enter "no" no


Recommending items similar to: Star Wars Rogue Squadron II: Rogue Leader

    Star Wars Rogue Squadron II: Rogue Leader has 284 reviews
    and a 4.3 average star rating

Here are the 10 recommended items for you based on your search parameters: 
15938

Tony Hawks' Pro Skater 3

    This item has 102 reviews
    and a 4.44 average star rating

Wave Race: Blue Storm

    This item has 99 reviews
    and a 3.89 average star rating

Gamecube Memory Card 59

    This item has 111 reviews
    and a 4.07 average star rating

Super Smash Bros Melee

    This item has 1060 reviews
    and a 4.59 average star rating

Luigi's Mansion - Gamecube

    This item has 501 reviews
    and a 3.92 average star rating

Super Monkey Ball

    This item has 161 reviews
    and a 4.31 average star rating

SSX Tricky Nintendo Gamecube

    This item has 120 reviews
    and a 4.48 average star rating

Star Wars: Rebel Strike - Rogue Squadron III

    This item has 104 reviews
    and a 3.9 average star rating